In [335]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [393]:
test_data= pd.read_csv('test_mSzZ8RL.csv')
train_data= pd.read_csv('train_s3TEQDk.csv')

In [394]:
train_data.set_index('ID',inplace=True)
train_data.head()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
ID,,,,,,,,,,
NNVBBKZB,Female,73,RG268,Other,X3,43,No,1045696,No,0
IDD62UNG,Female,30,RG277,Salaried,X1,32,No,581988,No,0
HD3DSEMC,Female,56,RG268,Self_Employed,X3,26,No,1484315,Yes,0
BF3NC7KV,Male,34,RG270,Salaried,X1,19,No,470454,No,0
TEASRWXV,Female,30,RG282,Salaried,X1,33,No,886787,No,0


In [402]:
test_data.fillna('Not Available',inplace=True)
train_data.fillna('Not Available',inplace=True)

In [403]:
#train_data.dropna(inplace=True)
train_data.isnull().sum()

Gender                 0
Age                    0
Region_Code            0
Occupation             0
Channel_Code           0
Vintage                0
Credit_Product         0
Avg_Account_Balance    0
Is_Active              0
Is_Lead                0
dtype: int64

In [404]:
test_data.set_index('ID',inplace=True)
test_data.head()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
ID,,,,,,,,,
VBENBARO,Male,29,RG254,Other,X1,25,Yes,742366,No
CCMEWNKY,Male,43,RG268,Other,X2,49,Yes,925537,No
VK3KGA9M,Male,31,RG270,Salaried,X1,14,No,215949,No
TT8RPZVC,Male,29,RG272,Other,X1,33,No,868070,No
SHQZEYTZ,Female,29,RG270,Other,X1,19,No,657087,No


In [405]:
train_data['Vintage']=train_data['Vintage'].floordiv(12)
test_data['Vintage']=test_data['Vintage'].floordiv(12)

In [406]:
from sklearn.preprocessing import StandardScaler 
sc = StandardScaler() 
num_cols=['Avg_Account_Balance']
train_data[num_cols] = sc.fit_transform(train_data[num_cols])
test_data[num_cols] = sc.transform(test_data[num_cols])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  """


In [407]:
def cat_pre_process(df,df_test,col,target):
    col=col
    target=target
    df1=pd.DataFrame(df[col].value_counts().reset_index())
    df2=pd.DataFrame(df[df[target]==1][col].value_counts().reset_index())
    df3=pd.merge(df1,df2,how='inner',on='index')
    df3['percent']=df3[df3.columns[-1]]/df3[df3.columns[-2]]*100
    transform = df3[['index','percent']]
    transform=dict(transform.to_dict('split')['data'])
    df[col]=df[col].map(transform)
    df_test[col]=df_test[col].map(transform)
    return df,df_test

In [408]:
cat_columns=['Gender', 'Age', 'Region_Code', 'Occupation', 'Channel_Code','Credit_Product','Is_Active','Vintage']
for col in cat_columns:
    train_data,test_data = cat_pre_process(train_data,test_data,col,'Is_Lead')

In [409]:
train_data.head()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
ID,,,,,,,,,,
NNVBBKZB,20.358116,23.385119,29.979963,24.495176,36.702468,19.378924,7.35884,-0.096968,20.815091,0
IDD62UNG,20.358116,7.771553,23.195073,15.966888,9.142097,13.827120,7.35884,-0.640630,20.815091,0
HD3DSEMC,20.358116,35.158296,29.979963,27.596495,36.702468,13.827120,7.35884,0.417279,28.296746,0
BF3NC7KV,26.515496,15.490977,14.430052,15.966888,9.142097,14.607186,7.35884,-0.771395,20.815091,0
TEASRWXV,20.358116,7.771553,19.214273,15.966888,9.142097,13.827120,7.35884,-0.283276,20.815091,0


In [410]:
test_data.head()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
ID,,,,,,,,,
VBENBARO,26.515496,8.024691,21.285395,24.495176,9.142097,13.827120,47.021743,-0.452599,20.815091
CCMEWNKY,26.515496,32.976632,29.979963,24.495176,32.789475,17.498914,47.021743,-0.237845,20.815091
VK3KGA9M,26.515496,8.116459,14.430052,15.966888,9.142097,14.607186,7.358840,-1.069782,20.815091
TT8RPZVC,26.515496,8.024691,21.991622,24.495176,9.142097,13.827120,7.358840,-0.305220,20.815091
SHQZEYTZ,20.358116,8.024691,14.430052,24.495176,9.142097,14.607186,7.358840,-0.552582,20.815091


In [411]:
train_data.columns[:-1]


Index(['Gender', 'Age', 'Region_Code', 'Occupation', 'Channel_Code', 'Vintage',
       'Credit_Product', 'Avg_Account_Balance', 'Is_Active'],
      dtype='object')

In [412]:
train_data.dropna(inplace=True)
train_data.isnull().sum()
xtrain=train_data[train_data.columns[:-1]]
ytrain=train_data[train_data.columns[-1]]


In [349]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(xtrain, ytrain)   
output=pd.DataFrame(pd.DataFrame(rf.predict_proba(test_data))[[1]])
output.to_csv('final')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [381]:
test_data.head()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
ID,,,,,,,,,
VBENBARO,26.515496,8.024691,21.285395,24.495176,9.142097,13.827120,31.495079,-0.452599,20.815091
CCMEWNKY,26.515496,32.976632,29.979963,24.495176,32.789475,17.498914,85.166240,-0.237845,20.815091
VK3KGA9M,26.515496,8.116459,14.430052,15.966888,9.142097,14.607186,7.358840,-1.069782,20.815091
TT8RPZVC,26.515496,8.024691,21.991622,24.495176,9.142097,13.827120,7.358840,-0.305220,20.815091
SHQZEYTZ,20.358116,8.024691,14.430052,24.495176,9.142097,14.607186,7.358840,-0.552582,20.815091


In [413]:
from xgboost import XGBClassifier
xgb = XGBClassifier(objective='binary:logistic')
xgb.fit(xtrain,ytrain)
output=pd.DataFrame(pd.DataFrame(rf.predict_proba(test_data))[[1]])
output.to_csv('XGB_output2')

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:41:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [391]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier()
lgb.fit(xtrain,ytrain)
output=pd.DataFrame(pd.DataFrame(rf.predict_proba(test_data))[[1]])
output.to_csv('LGB_Output')

C:\ProgramData\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
